In [40]:
#file setup

import cdflib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import math
import datetime
from scipy.fft import fftshift, fftfreq, fft
from scipy.integrate import trapezoid
from scipy.stats import sem
import calendar
import time
from plasmapy.formulary import lower_hybrid_frequency
from astropy import units as u

files = []
files.append(["../20170810data/mms1/edp/mms1_edp_brst_l2_dce_20170810121733_v3.0.0.cdf", "mms1_edp_dce_gse_brst_l2", "mms1_edp_epoch_brst_l2"])
files.append(["../20170810data/mms2/edp/mms2_edp_brst_l2_dce_20170810121733_v3.0.0.cdf", "mms2_edp_dce_gse_brst_l2", "mms2_edp_epoch_brst_l2"])
files.append(["../20170810data/mms3/edp/mms3_edp_brst_l2_dce_20170810121733_v3.0.0.cdf", "mms3_edp_dce_gse_brst_l2", "mms3_edp_epoch_brst_l2"])
files.append(["../20170810data/mms4/edp/mms4_edp_brst_l2_dce_20170810121733_v3.0.0.cdf", "mms4_edp_dce_gse_brst_l2", "mms4_edp_epoch_brst_l2"])

Bfiles = []
Bfiles.append(["../20170810data/mms1/fgm/mms1_fgm_brst_l2_20170810121733_v5.99.0.cdf", "mms1_fgm_b_gsm_brst_l2", "Epoch"])
Bfiles.append(["../20170810data/mms2/fgm/mms2_fgm_brst_l2_20170810121733_v5.99.0.cdf", "mms2_fgm_b_gsm_brst_l2", "Epoch"])
Bfiles.append(["../20170810data/mms3/fgm/mms3_fgm_brst_l2_20170810121733_v5.99.0.cdf", "mms3_fgm_b_gsm_brst_l2", "Epoch"])
Bfiles.append(["../20170810data/mms4/fgm/mms4_fgm_brst_l2_20170810121733_v5.99.0.cdf", "mms4_fgm_b_gsm_brst_l2", "Epoch"])

In [43]:
#function definitions

def readvariables(vartype):
    """Sort data into lists, only meant for rVariables and zVariables.
    For vartype input r or z."""
    returnlist = [] #returns this list
    for i in file.cdf_info().get(vartype + "Variables"):
        try:
            print(i + " " + str(file.varinq(i).get("Last_Rec"))) #Print name of variable and number of records
            returnlist.append([])
            for j in range(file.varinq(i).get("Last_Rec")):
                returnlist[file.cdf_info().get(vartype + "Variables").index(i)].append(file.varget(i,startrec=j,endrec=j+1).tolist())
                #Append data to the right list and right index
        except ValueError:
            print("No records found") #In case there are no records
            continue
    return returnlist

def filenames_get(name_list_file):
  '''
  Pulls list of filenames I'm using from the file where they are stored.
  Allows some flexibility
  Inputs:
      name_list_file- string which constains the full path to the file which
          contains a list of the full filename paths needed
  Outputs:
      name_list- list of strings which contain the full path to
          each file
  '''
  name_list=[]
  with open(name_list_file,"r") as name_file_obj: #read-only access
       for line in name_file_obj:
           line_clean =line.rstrip('\n') #removes newline chars from lines
           name_list.append(line_clean)
  return name_list

def get_cdf_var(filename,varnames):
  """
  pulls particular variables from a CDF
  note: if variable has more than one set of data (E.g. b-field with x,y,z
  components) it will be necessary to format the data by reshaping the array
  from a 1D to a 2D array
  (may find workaround/better way later)
  Inputs:
      filename- string of the complete path to the specified CDF file
      varnames- list of strings which contain the CDF variables that are
          to be extracted
  Outputs:
      data- list of numpy arrays containing the desired variables' data
  """
  cdf_file=cdflib.CDF(filename,varnames)
  data=[]
  for varname in varnames:
      var_data=np.array(cdf_file.varget(varname))
      data.append(var_data)
  return data

def getData(file, timeInterval, axis):
    CDFfile = cdflib.CDF(file[0])
    filename = file[0]
    varname = file[1]
    epochName = file[2]

    #generate time series
    raw_times = get_cdf_var(filename, [epochName])[0]
    times = []
    
    year = 2017
    month = 8
    day = 10
    hour = 12

    start_minute = timeInterval[0]
    start_sec = timeInterval[1]
    stop_minute = timeInterval[2]
    stop_sec = timeInterval[3]
    
    #Note that 'epoch' is 1/1/2000 12:00:00 (MMS epoch) J2000
    start_epoch_sec = calendar.timegm((year-30, month, day+1, hour-12, start_minute+1, start_sec+9))
    stop_epoch_sec = calendar.timegm((year-30, month, day+1, hour-12, stop_minute+1, stop_sec+9))
    start_index_epoch_sec = raw_times[0] / 1e9
    stop_index_epoch_sec = raw_times[-1] / 1e9
    time_between_measurements = (stop_index_epoch_sec - start_index_epoch_sec) / (len(raw_times) - 1)
    est_start_index = round((start_epoch_sec - start_index_epoch_sec) / time_between_measurements)
    est_stop_index = round((stop_epoch_sec - start_index_epoch_sec) / time_between_measurements)
    est_start_indexes = np.arange(est_start_index - 5, est_start_index + 5, 1).tolist()
    est_stop_indexes = np.arange(est_stop_index - 5, est_stop_index + 5, 1).tolist()
    
    for i in est_start_indexes:
        i_epoch_sec = round(raw_times[i] / 1e9)
        if i_epoch_sec == start_epoch_sec:
            start_index = i
    for j in est_stop_indexes:
        j_epoch_sec = round(raw_times[j] / 1e9)
        if j_epoch_sec == stop_epoch_sec:
            stop_index = j

    raw_data = get_cdf_var(filename, [varname])[0]

    if axis == "all":
        Ex = []
        Ey = []
        Ez = []
        
        for i in range(0,len(raw_data)):
            Ex.append(raw_data[i][0]) # x data
            Ey.append(raw_data[i][1]) # y data
            Ez.append(raw_data[i][2]) # z data

        Emag = vectorAdd(Ex, Ey, Ez)
        
        #data sets as arrays, #len = 2048
        x = raw_times[start_index:stop_index]
        # x = times
        y = Emag[start_index:stop_index]
        
    else:
        data = []
            
        coordinateSystem = {"x":0, "y":1, "z":2}
        coordinate = coordinateSystem.get(axis)

        for i in range(0,len(raw_data)):
            data.append(raw_data[i][coordinate])

        #data sets as arrays, #len = 2048
        x = raw_times[start_index:stop_index]
        # x = times
        y = data[start_index:stop_index]

    return x, y

def getFFTdata(x, y):
    N = len(x)
#     capT = (x[len(x)-1] - x[0])/1000000000
#     T = capT/N
    T = (x[1] - x[0])/1e9
#     print(T) ~7.8 millisecond timestep

    y2 = []
    for i in range(0, len(y)):
        y2.append(pow(y[i], 2))
        
    #average value
    sum = 0
    for i in range(0, N):
        sum += y2[i]

    average = sum/N
    
#     yf = fftshift(fft(y2-average))
    yf = fftshift(fft(y2))
    xf = fftshift(fftfreq(N, T))

    return xf, yf, N

def createTicks(time_interval, epoch, num_of_ticks=None):
    if num_of_ticks == None: #If not specified count 1 second per tick
        num_of_ticks = time_interval[3] - time_interval[1] + 1
    tick_indexes = []
    tick_values = np.linspace(time_interval[1], time_interval[3], num_of_ticks).tolist()

    distance_between_ticks = math.floor((len(epoch)/(num_of_ticks-1)))
    for x in range(num_of_ticks):
        if distance_between_ticks*x >= 1:
            tick_indexes.append(epoch[distance_between_ticks*x-1])
        else:
            tick_indexes.append(epoch[distance_between_ticks*x])
    return [tick_indexes, tick_values]

def powerSignal(x, y):
    N = len(x)
    
    sum = 0
    for i in range(0, N):
        sum += y[i]

    average = sum/N
    
    power = 0
    for j in range(0, N):
        power += pow(y[j]-average, 2)
        
    return power

def powerFFT(xf, yf):
        
    res = trapezoid(yf, xf)
    
    return res

def vectorAdd(Vx, Vy, Vz):
    N = len(Vx)
    Vmag = []
    
    for i in range(0, N):
        mag = math.sqrt(pow(Vx[i],2) + pow(Vy[i],2) + pow(Vz[i],2))
        Vmag.append(mag)
        
    return Vmag

def getLHfreq(period):
    MMS1x, MMS1y = getData(Bfiles[0], period, 'all')
    MMS2x, MMS2y = getData(Bfiles[1], period, 'all')
    MMS3x, MMS3y = getData(Bfiles[2], period, 'all')
    MMS4x, MMS4y = getData(Bfiles[3], period, 'all')
    
    N = len(MMS1x)
    
    B = []
    for i in range(0, N):
        avgB = (MMS1y[i] + MMS2y[i] + MMS3y[i] + MMS4y[i])/4
        B.append(avgB)
    
    LHfrequencies = []
    avgLHfreq = 0

    for i in range(0, N):
        fLH = lower_hybrid_frequency(B[i]*u.nT, n_i=0.17*u.cm**-3, ion="p+")
        LHfrequencies.append(fLH.value/(2*math.pi))
        avgLHfreq += fLH.value

    
    LHmean = avgLHfreq/(2*N*math.pi)
    LHsem = sem(LHfrequencies)
    
    return LHmean, LHsem

def FGManalysis(period, axis, ppObj, title, FFTtitle):
    
    coordinateLabels = {"x":"Ex", "y":"Ey", "z":"Ez", "all":"|E|"}
    
    MMS1x, MMS1y = getData(files[0], period, axis)
    MMS2x, MMS2y = getData(files[1], period, axis)
    MMS3x, MMS3y = getData(files[2], period, axis)
    MMS4x, MMS4y = getData(files[3], period, axis)

    #Raw Signal
    fig1 = plt.figure(1)

    fig1.autofmt_xdate()
    
    ticks = createTicks(period, MMS1x)
    plt.xticks(ticks[0], ticks[1])  
    
    s1 = plt.plot(MMS1x, MMS1y, '-k')
    s2 = plt.plot(MMS2x , MMS2y, '-r')
    s3 = plt.plot(MMS3x , MMS3y, '-g')
    s4 = plt.plot(MMS4x , MMS4y, '-b')
    
    plt.legend(labels = ('MMS1', 'MMS2', 'MMS3', 'MMS4'), loc = 'lower right')
    plt.title(title)
    plt.xlabel('2017-08-10 12:18 UTC, Epoch (s)')
    plt.ylabel(coordinateLabels.get(axis) + ' Field (nT)')

    MMS1xf, MMS1yf, MMS1N = getFFTdata(MMS1x, MMS1y)
    MMS2xf, MMS2yf, MMS2N = getFFTdata(MMS2x, MMS2y)
    MMS3xf, MMS3yf, MMS3N = getFFTdata(MMS3x, MMS3y)
    MMS4xf, MMS4yf, MMS4N = getFFTdata(MMS4x, MMS4y)

    #FFT
    fig2 = plt.figure(2)
    plt.plot(MMS1xf, 1.0/MMS1N * np.abs(MMS1yf), '-k')
    plt.plot(MMS2xf, 1.0/MMS2N * np.abs(MMS2yf),'-r')
    plt.plot(MMS3xf, 1.0/MMS3N * np.abs(MMS3yf), '-g')
    plt.plot(MMS4xf, 1.0/MMS4N * np.abs(MMS4yf), '-b')
    
    avgLH, semLH = getLHfreq(period)
    plt.axvline(x=avgLH, color='m', linestyle='--')
    plt.axvspan(avgLH-semLH, avgLH+semLH, alpha=0.5, color='y')

    
    plt.yscale('log')
    plt.legend(labels = ('MMS1', 'MMS2', 'MMS3', 'MMS4', 'Avg. LH Freq.', 'Standard Error'), loc = 'upper right')
    plt.title(FFTtitle)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Wave Energy ' + coordinateLabels.get(axis) + '^2 (nT^2)')

#     #set FFT window
    plt.xlim([0, 10])
#     plt.ylim([10**-2,10**2])

    plt.grid()

    pdf.savefig(fig1)
    pdf.savefig(fig2)
    plt.close('all')

period = [] #start min, start sec, stop min, stop sec

In [44]:
#main plotting

pdf = PdfPages("./Results/fgmEResults.pdf")

# FGManalysis([18, 29, 18, 37], "all", pdf, "FGM |E| Field Plot - Full Period", "FGM |E| Field FFT - Full Period")
# # FGManalysis([18, 29, 18, 33], "all", pdf, "FGM |B| Field Plot - 1/2 Period", "FGM |B| Field FFT - 1/2 Period")
# # FGManalysis([18, 33, 18, 37], "all", pdf, "FGM |B| Field Plot - 2/2 Period", "FGM |B| Field FFT - 2/2 Period")
# FGManalysis([18, 29, 18, 31], "all", pdf, "FGM |E| Field Plot - 1/4 Period", "FGM |E| Field FFT - 1/4 Period")
# FGManalysis([18, 31, 18, 33], "all", pdf, "FGM |E| Field Plot - 2/4 Period", "FGM |E| Field FFT - 2/4 Period")
# FGManalysis([18, 33, 18, 35], "all", pdf, "FGM |E| Field Plot - 3/4 Period", "FGM |E| Field FFT - 3/4 Period")
# FGManalysis([18, 35, 18, 37], "all", pdf, "FGM |E| Field Plot - 4/4 Period", "FGM |E| Field FFT - 4/4 Period")

# FGManalysis([18, 29, 18, 37], "x", pdf, "FGM Ex Field Plot - Full Period", "FGM Ex Field FFT - Full Period")
# # FGManalysis([18, 29, 18, 33], "x", pdf, "FGM Bx Field Plot - 1/2 Period", "FGM Bx Field FFT - 1/2 Period")
# # FGManalysis([18, 33, 18, 37], "x", pdf, "FGM Bx Field Plot - 2/2 Period", "FGM Bx Field FFT - 2/2 Period")
# FGManalysis([18, 29, 18, 31], "x", pdf, "FGM Bx Field Plot - 1/4 Period", "FGM Bx Field FFT - 1/4 Period")
# FGManalysis([18, 31, 18, 33], "x", pdf, "FGM Bx Field Plot - 2/4 Period", "FGM Bx Field FFT - 2/4 Period")
# FGManalysis([18, 33, 18, 35], "x", pdf, "FGM Bx Field Plot - 3/4 Period", "FGM Bx Field FFT - 3/4 Period")
# FGManalysis([18, 35, 18, 37], "x", pdf, "FGM Bx Field Plot - 4/4 Period", "FGM Bx Field FFT - 4/4 Period")

# FGManalysis([18, 29, 18, 37], "y", pdf, "FGM Ey Field Plot - Full Period", "FGM Ey Field FFT - Full Period")
# # FGManalysis([18, 29, 18, 33], "y", pdf, "FGM By Field Plot - 1/2 Period", "FGM By Field FFT - 1/2 Period")
# # FGManalysis([18, 33, 18, 37], "y", pdf, "FGM By Field Plot - 2/2 Period", "FGM By Field FFT - 2/2 Period")
# FGManalysis([18, 29, 18, 31], "y", pdf, "FGM By Field Plot - 1/4 Period", "FGM By Field FFT - 1/4 Period")
# FGManalysis([18, 31, 18, 33], "y", pdf, "FGM By Field Plot - 2/4 Period", "FGM By Field FFT - 2/4 Period")
# FGManalysis([18, 33, 18, 35], "y", pdf, "FGM By Field Plot - 3/4 Period", "FGM By Field FFT - 3/4 Period")
# FGManalysis([18, 35, 18, 37], "y", pdf, "FGM By Field Plot - 4/4 Period", "FGM By Field FFT - 4/4 Period")

FGManalysis([18, 29, 18, 37], "z", pdf, "FGM Ez Field Plot - Full Period", "FGM Ez Field FFT - Full Period")
# # FGManalysis([18, 29, 18, 33], "z", pdf, "FGM Bz Field Plot - 1/2 Period", "FGM Bz Field FFT - 1/2 Period")
# # FGManalysis([18, 33, 18, 37], "z", pdf, "FGM Bz Field Plot - 2/2 Period", "FGM Bz Field FFT - 2/2 Period")
FGManalysis([18, 29, 18, 31], "z", pdf, "FGM Ez Field Plot - 1/4 Period", "FGM Ez Field FFT - 1/4 Period")
FGManalysis([18, 31, 18, 33], "z", pdf, "FGM Ez Field Plot - 2/4 Period", "FGM Ez Field FFT - 2/4 Period")
FGManalysis([18, 33, 18, 35], "z", pdf, "FGM Ez Field Plot - 3/4 Period", "FGM Ez Field FFT - 3/4 Period")
FGManalysis([18, 35, 18, 37], "z", pdf, "FGM Ez Field Plot - 4/4 Period", "FGM Ez Field FFT - 4/4 Period")

pdf.close()

print("Done.")

Done.
